In [272]:
#import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization

In [273]:
#Load data
train_data=pd.read_csv('train.csv')
test_data=pd.read_csv('test.csv')

In [274]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1459 non-null   int64  
 1   MSSubClass     1459 non-null   int64  
 2   MSZoning       1455 non-null   object 
 3   LotFrontage    1232 non-null   float64
 4   LotArea        1459 non-null   int64  
 5   Street         1459 non-null   object 
 6   Alley          107 non-null    object 
 7   LotShape       1459 non-null   object 
 8   LandContour    1459 non-null   object 
 9   Utilities      1457 non-null   object 
 10  LotConfig      1459 non-null   object 
 11  LandSlope      1459 non-null   object 
 12  Neighborhood   1459 non-null   object 
 13  Condition1     1459 non-null   object 
 14  Condition2     1459 non-null   object 
 15  BldgType       1459 non-null   object 
 16  HouseStyle     1459 non-null   object 
 17  OverallQual    1459 non-null   int64  
 18  OverallC

In [275]:
#Check the columns in the data
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [276]:
train_data.isnull().sum()

Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
                ... 
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
SalePrice          0
Length: 81, dtype: int64

In [277]:
#Select relevant features (square footage, no of bedrooms and bathrooms)
features=['GrLivArea', 'BedroomAbvGr', 'FullBath']
target='SalePrice'

In [278]:
#Extract features and target for training
X_train_full=train_data[features]
Y_train_full=train_data[target]

In [279]:
#Extract features for test set
X_test_full=test_data[features]

In [280]:
#Handle missing values
X_train_full.fillna(0, inplace=True)
X_test_full.fillna(0, inplace=True)

C:\Users\Admin\AppData\Local\Temp\ipykernel_43484\284850582.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_full.fillna(0, inplace=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_43484\284850582.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_full.fillna(0, inplace=True)


import xgboost
classifier=xgboost.XGBRegressor()
classifier.fit(X_train_full,Y_train_full)

Predicting and choosing the Algorithm

import xgboost
regressor=xgboost.XGBRegressor()

booster=['gbtree','bglinear']
base_score=[0.25,0.5,0.75,1]

#Hyper Parameter Optimization

n_estimators = [100, 500, 900, 1100, 1500]
max_depth = [2, 3, 5, 10, 15]
booster=['gbtree','gblinear']
learning_rate=[0.05,0.1,0.15,0.20]
min_child_weight=[1,2,3,4]

#Define the grid of hyperparameters to search
hyperparameter_grid = {
    'n_estimators': n_estimators,
    'max_depth': max_depth,
    'learning_rate': learning_rate,
    'min_child_rate': min_child_weight,
    'booster': booster,
    'base_score': base_score
}

from sklearn.model_selection import RandomizedSearchCV

#Set up the random search with 4-fold cross validation
random_cv = RandomizedSearchCV(estimator=regressor,
                               param_distributions=hyperparameter_grid,
                               cv=5, n_iter=50,
                               scoring = 'neg_mean_absolute_error',n_jobs = 4,
                               verbose = 5,
                               return_train_score = True,
                               random_state=42)

random_cv.fit(X_train_full,Y_train_full)

random_cv.best_estimator_

regressor=xgboost.XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=2, max_leaves=None,
             min_child_rate=1, min_child_weight=None,
             monotone_constraints=None, multi_strategy=None, n_estimators=100,
             n_jobs=None, num_parallel_tree=None)

regressor.fit(X_train_full,Y_train_full)

In [281]:
#Normalize features
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train_full)
X_test=scaler.fit_transform(X_test_full)

In [282]:
#Split training data for validation
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train_full, test_size=0.2, random_state=42)

Building the ANN Model

In [ ]:
from keras.regularizers import l2

In [ ]:
model = Sequential()

#Creating the input layer
model.add(Dense(units=50, kernel_initializer = 'he_uniform',kernel_regularizer=l2(0.01),activation='relu', input_dim=X_train.shape[1]))  
BatchNormalization()
Dropout(0.2)

#Creating hidden layer 1
model.add(Dense(units=25, kernel_initializer = 'he_uniform',kernel_regularizer=l2(0.01),activation='relu'))  
BatchNormalization()
Dropout(0.2)

#Creating hidden layer 2
model.add(Dense(units=50, kernel_initializer = 'he_uniform',kernel_regularizer=l2(0.01),activation='relu'))  
BatchNormalization()
Dropout(0.2)

#Output layer
model.add(Dense(units=1, kernel_initializer = 'he_uniform',activation='linear'))  

c:\Program Files\Python39\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [284]:
# Compile the Model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])  # MSE = loss, MAE = metric

In [285]:
# Train the Model
history = model.fit(
    X_train, Y_train,
    validation_data=(X_val, Y_val),
    epochs=1500,  
    batch_size=32,
    verbose=1
)

Epoch 1/1500
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 41473929216.0000 - mae: 186205.2500 - val_loss: 39649779712.0000 - val_mae: 178828.9531
Epoch 2/1500
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 39624085504.0000 - mae: 182936.0312 - val_loss: 39633973248.0000 - val_mae: 178787.9531
Epoch 3/1500
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 39249485824.0000 - mae: 181468.8594 - val_loss: 39580803072.0000 - val_mae: 178648.8281
Epoch 4/1500
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 39244640256.0000 - mae: 181551.6094 - val_loss: 39441551360.0000 - val_mae: 178285.9062
Epoch 5/1500
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 38216593408.0000 - mae: 179701.5625 - val_loss: 39142547456.0000 - val_mae: 177506.8594
Epoch 6/1500
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 39444189184.0000 - mae: 181557.7656 - val_loss: 38581702656.0000 - val_mae: 176044.5625
Epoch 7/1500
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 37705388032.0000 - mae: 178137.9062 - val_loss: 3764

In [286]:
# Evaluate Model Performance
val_loss, val_mae = model.evaluate(X_val, Y_val, verbose=0)
print(f"Validation Loss (MSE): {val_loss}")
print(f"Validation MAE: {val_mae}")

Validation Loss (MSE): 2583727872.0
Validation MAE: 33721.95703125


In [287]:
# Predict on Test Data
test_predictions = model.predict(X_test)

46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


In [288]:
# Save Predictions to a CSV File
output = pd.DataFrame({'Id': test_data['Id'], 'SalePrice': test_predictions.flatten()})
output.to_csv('house_price_predictions.csv', index=False)

In [289]:
# Evaluate on Entire Training Data
Y_train_pred = model.predict(X_train)
mae = mean_absolute_error(Y_train, Y_train_pred)
mse = mean_squared_error(Y_train, Y_train_pred)
r2 = r2_score(Y_train, Y_train_pred)

print(f"Training Performance - MAE: {mae}, MSE: {mse}, R-squared: {r2}")

37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Training Performance - MAE: 32726.09005110231, MSE: 2364487514.2576265, R-squared: 0.6035765709554446


In [290]:
# Example Prediction on Custom Data
custom_data = [[2000, 3, 2]]  # Example: 2000 sqft, 3 bedrooms, 2 bathrooms
custom_data_scaled = scaler.transform(custom_data)
predicted_price = model.predict(custom_data_scaled)
print(f"Predicted Price for Custom Data: {predicted_price[0][0]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
Predicted Price for Custom Data: 253856.0625


C:\Users\Admin\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
